# 10-1 간단한 함수 만들기
## 제곱 함수와 n 제곱 함수 만들기

In [1]:
def my_sq(x):
    return x ** 2

In [2]:
def my_exp(x, n):
    return x ** n

In [3]:
print(my_sq(4))

16


In [4]:
print(my_exp(2, 4))

16


# 10-2 apply 메서드 활용하기
## 시리즈와 데이터프레임에 apply 메서드 사용하기

### 1. 시리즈와 apply 메서드

In [5]:
import pandas as pd

df = pd.DataFrame({'a': [10, 20, 30], 'b': [20, 30, 40]}) 

print(df)

    a   b
0  10  20
1  20  30
2  30  40


apply 메서드에 제곱함수의 이름(my_sq)을 전달해 시리즈의 모든 데이터에 제곱함수를 적용한 것

#### 함수인자 하나인 경우 

In [6]:
sq = df['a'].apply(my_sq) #인자값 생략
print(sq)

0    100
1    400
2    900
Name: a, dtype: int64


#### 함수인자 2개 이상인 경우

In [7]:
ex = df['a'].apply(my_exp, n=2) 
print(ex)

0    100
1    400
2    900
Name: a, dtype: int64


In [8]:
ex = df['a'].apply(my_exp, n=3) 
print(ex)

0     1000
1     8000
2    27000
Name: a, dtype: int64


### 5. 데이터 프레임과 apply 메서드

In [9]:
df = pd.DataFrame({'a': [10, 20, 30], 'b': [20, 30, 40]}) 
print(df)

    a   b
0  10  20
1  20  30
2  30  40


In [10]:
def print_me(x): 
    print(x)

In [11]:
print(df.apply(print_me, axis=0)) #함수를 열방향으로 적용(default), 행방향으로 적용하고 싶을 때는 axis=1

0    10
1    20
2    30
Name: a, dtype: int64
0    20
1    30
2    40
Name: b, dtype: int64
a    None
b    None
dtype: object


In [12]:
print(df['a'])

0    10
1    20
2    30
Name: a, dtype: int64


In [13]:
print(df['b'])

0    20
1    30
2    40
Name: b, dtype: int64


세개의인자를 입력받아 평균을 계산하는 함수

In [14]:
def avg_3(x, y, z):
    return (x + y + z) / 3

In [15]:
print(df.apply(avg_3))

TypeError: ("avg_3() missing 2 required positional arguments: 'y' and 'z'", 'occurred at index a')

In [16]:
def avg_3_apply(col):
    x = col[0] 
    y = col[1] 
    z = col[2] 
    return (x + y + z) / 3


print(df.apply(avg_3_apply))

a    20.0
b    30.0
dtype: float64


In [17]:
df.shape

(3, 2)

행의 개수를 모르는 경우는 이렇게 표현한다.

In [18]:
def avg_3_apply(col):
    sum = 0
    for item in col:
         sum += item
    return sum / df.shape[0]

In [19]:
print(df.apply(avg_3_apply))

a    20.0
b    30.0
dtype: float64


In [20]:
#행 별로 평균 구해보기 
def avg_3_apply(row):
    sum = 0
    for item in row:
         sum += item
    return sum / df.shape[1]

In [23]:
print(df.apply(avg_3_apply,axis=1))

0    15.0
1    25.0
2    35.0
dtype: float64


# 10-3 apply 메서드 사용하기 - 고급
## 데이터프레임의 누락값을 처리한 다음 apply 메서드 사용하기

### 1. 데이터프레임의 누락값 처리하기 ― 열 방향

In [24]:
import seaborn as sns

titanic = sns.load_dataset("titanic")

In [25]:
print(titanic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            714 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB
None


#### 열별로 누락값 개수 찾기

In [27]:
import numpy as np

def count_missing(vec):
    null_vec = pd.isnull(vec)  #누락값 찾기
    null_count = np.sum(null_vec)  #누락값 갯수 세기
    return null_count

In [28]:
cmis_col = titanic.apply(count_missing)
print(cmis_col)

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64


#### 열별로 결측값 비율 찾기 

In [29]:
def prop_missing(vec):
    num = count_missing(vec)
    dem = vec.size
    return num / dem

In [30]:
pmis_col = titanic.apply(prop_missing)
print(pmis_col)

survived       0.000000
pclass         0.000000
sex            0.000000
age            0.198653
sibsp          0.000000
parch          0.000000
fare           0.000000
embarked       0.002245
class          0.000000
who            0.000000
adult_male     0.000000
deck           0.772166
embark_town    0.002245
alive          0.000000
alone          0.000000
dtype: float64


In [31]:
def prop_complete(vec):
    return 1 - prop_missing(vec)

### 8. 데이터프레임의 누락값을 처리하기 ― 행 방뱡

In [32]:
cmis_row = titanic.apply(count_missing, axis=1)
pmis_row = titanic.apply(prop_missing, axis=1)
pcom_row = titanic.apply(prop_complete, axis=1)

print(cmis_row.head())

0    1
1    0
2    1
3    0
4    1
dtype: int64


In [33]:
print(pmis_row.head())

0    0.066667
1    0.000000
2    0.066667
3    0.000000
4    0.066667
dtype: float64


In [34]:
print(pcom_row.head())

0    0.933333
1    1.000000
2    0.933333
3    1.000000
4    0.933333
dtype: float64


In [35]:
#titanic에 num_missing이라는 컬럼을 새로 만들어 각 행마다의 결측값을 세준다.
titanic['num_missing']=cmis_row

In [37]:
print(titanic.head())

   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  num_missing  
0    man        True  NaN  Southampton    no  False            1  
1  woman       False    C    Cherbourg   yes  False            0  
2  woman       False  NaN  Southampton   yes   True            1  
3  woman       False    C  Southampton   yes  False            0  
4    man        True  NaN  Southampton    no   True            1  


In [41]:
#결측값이 2개이상인 행들에 대해서만 뽑아낸다.
titanic.loc[titanic.num_missing>1,:]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,num_missing
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True,2
17,1,2,male,NaN,0,0,13.0000,S,Second,man,True,NaN,Southampton,yes,True,2
19,1,3,female,NaN,0,0,7.2250,C,Third,woman,False,NaN,Cherbourg,yes,True,2
26,0,3,male,NaN,0,0,7.2250,C,Third,man,True,NaN,Cherbourg,no,True,2
28,1,3,female,NaN,0,0,7.8792,Q,Third,woman,False,NaN,Queenstown,yes,True,2
29,0,3,male,NaN,0,0,7.8958,S,Third,man,True,NaN,Southampton,no,True,2
32,1,3,female,NaN,0,0,7.7500,Q,Third,woman,False,NaN,Queenstown,yes,True,2
36,1,3,male,NaN,0,0,7.2292,C,Third,man,True,NaN,Cherbourg,yes,True,2
42,0,3,male,NaN,0,0,7.8958,C,Third,man,True,NaN,Cherbourg,no,True,2
45,0,3,male,NaN,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,2
